# Matrix Factorization

We will implement matrix factorization here, both manually and maybe through TensorFlow.

## Theory Behind Matrix Factorization: Problem Statement

We will give a brief rundown, nothing too complicated.

The main **mathematical** statement that we will tackle with matrix factorization is as follows:

We have some matrix $A$ (this could be a tabular dataset or anything else). Now the *caveat* is that $A$ has **missing entries** that do not know about. We want to find two matrices $U$ and $V$, so that their product:

$$U \cdot V^{T} \approx A$$

So really, in terms of an optimization problem, we want to minimize each entry of the matrix $E$:

$$E = A - U\cdot V^{T}$$

It really helps to write this in index notation to represent each component so that we are not dealing with matrices directly. Let $E_{ij} := e_{ij}$, then, writing out the above expression in matrix form, we see that:

$$e_{ij} = a_{ij} - \sum_{k}u_{ik}v_{jk}^{T} = a_{ij} - \sum_{k}u_{ik}v_{kj}$$

We are now only working with real numbers! We can now adopt the traditional MSE loss for each entry.

$$e_{ij}^{2} = \left(a_{ij} - \sum_{k}u_{ik}v_{kj}\right)^{2}$$

or to make it look simpler, we can write $\hat{a}_{ij} = \sum_{k}u_{ik}v_{kj}$, and we now have:

$$e_{ij}^{2} = (a_{ij} - \hat{a}_{ij})^{2}$$

We are now minimizing this expression for all $(i,j) \in rows(A), cols(A))$. **For all intents and purposes**, this is all we really need to know to continue.

## Theory of Matrix Factorization: Machine Learning

However, we can continue our derivation of our equations, as this will help us in our implementation. We take the gradient (derivative) of our loss above, with respect to the nonzero components of both $U$ and $V$ (**Notice that we are not training weights and biases, but just the entries of the matrices $U$ and $V$!!!**):

$$\frac{\partial e_{ij}^{2}}{\partial u_{ik}} = \frac{\partial e_{ij}^{2}}{\partial e_{ij}}\frac{\partial e_{ij}}{\partial u_{ik}} = -2e_{ij}v_{kj} $$

Likewise, for the other component:

$$\frac{\partial e_{ij}^{2}}{\partial v_{ik}} = -2e_{ij}u_{ik}$$

We now do the **optimization step** (i.e. the *Gradient Descent Step*):

$$u_{ik} \longmapsto u_{ik} - \lambda \frac{\partial e_{ij}^{2}}{\partial u_{ik}} = u_{ik} + 2\lambda e_{ij}v_{kj}$$

$$v_{kj} \longmapsto v_{kj} - \lambda \frac{\partial e_{ij}^{2}}{\partial v_{kj}} = v_{kj} + 2\lambda e_{ij}u_{ik}$$

## Business Use of Matrix Factorization

The business problem we are trying to solve is the following:

We are given a table of **customers** and **products** they interacted with, however, some customers do not leave ratings on the product that they interacted with. In this case, how do we find these missing ratings?

Furthermore, let us say that we track certain features for the customers and products. For example:

Let $U$ denote the customers, and let $V$ denote items bought at a food mart. Let us say that $U$, $V$ contains **2 features**, sweet and savory, and let $A$ be the rating of sweet or savory items (from 1-5) by a customer that purchased them. Sweet and savory are what we refer to as **latent features**, and they relate customers to products as they are keys that merge customers and products. 

Now assume $A$ is the following customer-product interaction table...

$$A = \begin{pmatrix} 
4 & 0 & 3 & 2 & 2 & 1\\[3pt]
0 & 5 & 3 & 4 & 5 & 1\\[3pt]
5 & 5 & 0 & 0 & 3 & 5\\[3pt]
3 & 3 & 4 & 3 & 2 & 0\\[3pt]
2 & 4 & 2 & 4 & 5 & 1\\[3pt]
0 & 0 & 5 & 3 & 3 & 4
\end{pmatrix}$$

Then we want to find $U$ and $V$ such that:

- $U$ encodes the latent features (sweet or savory) of the customers, and $V$ does the same for the products.
- $U \cdot V^{T} \approx A$.

Thus, we are looking for $U$, $V$ such that:

$$A \approx U\cdot V^{T}$$

Now refer to the previous section for the mathematical details of our machine learning problem.


## Manual Implementation of Matrix Factorization

By "manual implementation", we mean *implementation in python with only numpy*. This is possible because we are using the following loss:

$$\mathrm{Loss} = \mathrm{MSE} + \mathrm{Regularization}$$

This loss goes component-by-component:

$$e_{ij}^{2} = (a_{ij} - \hat{a}_{ij})^{2}$$

The total loss is

$$\hat{L} = \sum_{i,j} e_{ij}^{2}$$

We also add a regularization term:

$$L = \hat{L} + \sum_{k=1}^{N}\beta\left(u_{ik}^{2} + v_{jk}^{2}\right)$$

For $N$ latent features. This is to prevent overfitting.

So we have a closed-form expression for the gradient of the loss in this case. If we wanted to implement more complicated loss functions, we will likely need to use an automatic differentiation package.

### Matrix Factorization 1: A Simple Implementation

In [1]:
import numpy as np

import timeit as t

In [2]:
#ARGUMENTS
# A - ground truth matrix
# U - product 1
# V - product 2
# number of features
# epochs - number of epochs to train
# lambda - learning rate
# beta - regularization parameter

def matrix_factorization(A, U, V, k, steps, lmbda=0.0002, beta = 0.02):
    V = V.T

    for epoch in range(steps):
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                # we are only looking to minimize the loss on the already-filled entries
                if A[i][j] > 0:
                    # error
                    eij = A[i][j] - np.dot(U[i, :], V[:, j])
                    
                    for K in range(k):
                        # gradient descent step
                        U[i][K] = U[i][K] + lmbda * (2*eij * V[K][j] - beta*U[i][K])
                        V[K][j] = V[K][j] + lmbda * (2*eij * U[i][K] - beta*V[K][j])
        
        # e is our loss (i.e. our e_{ij}^{2})
        e = 0
        
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                if A[i][j] > 0:
                    # update the loss
                    e = e + (A[i][j] - np.dot(U[i,:], V[:,j]))**2
                    
                    for K in range(k):
                        # put regularization term on loss so it doesn't overfit on training data
                        e = e + (beta/2) * (U[i][K]**2 + V[K][j]**2)
                     
        # give print output of training (print only every 10 epochs)
        if epoch % 10 == 0:
            print(f'Epoch {epoch + 1} : Total Loss {e:.3f}')
        if e < 0.001:
            break
    return U, V.T

In [3]:
A = np.array([
    [5,3,2,1,0],
    [3,0,4,2,1],
    [4,0,0,1,0],
    [0,0,2,4,0],
    [5,3,0,0,1]
])

m = A.shape[0]
n = A.shape[1]
user_features = 2

np.random.seed(24)

# generate matrices with entries of random numbers 0 - 1
U = np.random.rand(m, user_features)
V = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U)
print('V is:\n', V)

# perform factorization

U_hat, V_hat = matrix_factorization(A, U, V, user_features, 200)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A

print('A_hat is:\n', np.dot(U_hat, V_hat.T))

U is:
 [[0.9600173  0.69951205]
 [0.99986729 0.2200673 ]
 [0.36105635 0.73984099]
 [0.99645573 0.31634698]
 [0.13654458 0.38398001]]
V is:
 [[0.32051928 0.36641475]
 [0.70965156 0.90014243]
 [0.53411544 0.24729376]
 [0.67180656 0.56172911]
 [0.54255988 0.8934476 ]]
Epoch 1 : Total Loss 99.275
Epoch 11 : Total Loss 96.767
Epoch 21 : Total Loss 94.202
Epoch 31 : Total Loss 91.584
Epoch 41 : Total Loss 88.918
Epoch 51 : Total Loss 86.207
Epoch 61 : Total Loss 83.459
Epoch 71 : Total Loss 80.680
Epoch 81 : Total Loss 77.878
Epoch 91 : Total Loss 75.059
Epoch 101 : Total Loss 72.233
Epoch 111 : Total Loss 69.408
Epoch 121 : Total Loss 66.595
Epoch 131 : Total Loss 63.801
Epoch 141 : Total Loss 61.036
Epoch 151 : Total Loss 58.311
Epoch 161 : Total Loss 55.633
Epoch 171 : Total Loss 53.012
Epoch 181 : Total Loss 50.455
Epoch 191 : Total Loss 47.970
U_hat is:
 [[1.2427563  0.99404657]
 [1.32958304 0.4449768 ]
 [0.53087359 0.91554011]
 [1.23935835 0.47920855]
 [0.51268872 0.8134178 ]]
V_hat is

This does not look good after 200 epochs! Let's try more epochs, and if that doesn't work, let's increase the learning rate.

In [4]:
U_hat, V_hat = matrix_factorization(A, U, V, user_features, 5000)

print('A_hat is:\n', np.dot(U_hat, V_hat.T))

Epoch 1 : Total Loss 45.564
Epoch 11 : Total Loss 43.244
Epoch 21 : Total Loss 41.013
Epoch 31 : Total Loss 38.877
Epoch 41 : Total Loss 36.838
Epoch 51 : Total Loss 34.898
Epoch 61 : Total Loss 33.060
Epoch 71 : Total Loss 31.323
Epoch 81 : Total Loss 29.687
Epoch 91 : Total Loss 28.150
Epoch 101 : Total Loss 26.711
Epoch 111 : Total Loss 25.366
Epoch 121 : Total Loss 24.114
Epoch 131 : Total Loss 22.950
Epoch 141 : Total Loss 21.870
Epoch 151 : Total Loss 20.871
Epoch 161 : Total Loss 19.947
Epoch 171 : Total Loss 19.095
Epoch 181 : Total Loss 18.310
Epoch 191 : Total Loss 17.587
Epoch 201 : Total Loss 16.923
Epoch 211 : Total Loss 16.312
Epoch 221 : Total Loss 15.751
Epoch 231 : Total Loss 15.236
Epoch 241 : Total Loss 14.763
Epoch 251 : Total Loss 14.328
Epoch 261 : Total Loss 13.929
Epoch 271 : Total Loss 13.563
Epoch 281 : Total Loss 13.225
Epoch 291 : Total Loss 12.914
Epoch 301 : Total Loss 12.628
Epoch 311 : Total Loss 12.364
Epoch 321 : Total Loss 12.120
Epoch 331 : Total Los

Epoch 2761 : Total Loss 5.540
Epoch 2771 : Total Loss 5.539
Epoch 2781 : Total Loss 5.538
Epoch 2791 : Total Loss 5.537
Epoch 2801 : Total Loss 5.536
Epoch 2811 : Total Loss 5.535
Epoch 2821 : Total Loss 5.534
Epoch 2831 : Total Loss 5.533
Epoch 2841 : Total Loss 5.532
Epoch 2851 : Total Loss 5.531
Epoch 2861 : Total Loss 5.531
Epoch 2871 : Total Loss 5.530
Epoch 2881 : Total Loss 5.529
Epoch 2891 : Total Loss 5.528
Epoch 2901 : Total Loss 5.527
Epoch 2911 : Total Loss 5.526
Epoch 2921 : Total Loss 5.526
Epoch 2931 : Total Loss 5.525
Epoch 2941 : Total Loss 5.524
Epoch 2951 : Total Loss 5.523
Epoch 2961 : Total Loss 5.523
Epoch 2971 : Total Loss 5.522
Epoch 2981 : Total Loss 5.521
Epoch 2991 : Total Loss 5.521
Epoch 3001 : Total Loss 5.520
Epoch 3011 : Total Loss 5.519
Epoch 3021 : Total Loss 5.519
Epoch 3031 : Total Loss 5.518
Epoch 3041 : Total Loss 5.517
Epoch 3051 : Total Loss 5.517
Epoch 3061 : Total Loss 5.516
Epoch 3071 : Total Loss 5.516
Epoch 3081 : Total Loss 5.515
Epoch 3091

After training for long enough, we see that the existing interactions are predicted quite well. To see the average error of each entry in $A_{hat}$ corresponding to NONZEROS in $A$, we can just use the following:

In [5]:
A_hat = np.dot(U_hat, V_hat.T)
A_hat

array([[4.98447014, 2.97009182, 1.54598166, 1.50219606, 0.9902487 ],
       [2.99605858, 2.12761136, 3.17161116, 2.87072417, 0.95654322],
       [3.98158649, 2.34293891, 1.04127825, 1.03001406, 0.75980498],
       [4.52329661, 2.9474781 , 3.05471784, 2.81272953, 1.16479036],
       [4.96573884, 3.01061644, 1.87870719, 1.79363369, 1.0410779 ]])

In [6]:
A

array([[5, 3, 2, 1, 0],
       [3, 0, 4, 2, 1],
       [4, 0, 0, 1, 0],
       [0, 0, 2, 4, 0],
       [5, 3, 0, 0, 1]])

### Write a Method to Evaluate our Output

Since this is a regression task, we will be evaluating the performance of our model through (root) mean squared error.

In [7]:
# have function to evaluate total loss in our prediction

def evaluation_metrics(A, A_hat, return_nonzero_indices = False, return_rmse = True):
    # find all nonzero indices of ground truth matrix
    nonzero_ind = [(i,j) for i in range(A.shape[0]) for j in range(A.shape[1]) if A[i,j] > 0]
    
    # compute the losses in each ground truth matrix
    avg_error = sum([(A_hat[i][j] - A[i][j])**2 for i, j in nonzero_ind])/len(nonzero_ind)
    rmse = np.sqrt(avg_error)
    
    # set a flag for if we want to return the nonzero indices or not
    li_results = []
    if return_rmse:
        li_results.append(avg_error)
        li_results.append(rmse)
    else:
        li_results.append(avg_error)
    
    if return_nonzero_indices:
        li_results.append(nonzero_indices)
    else:
        pass
    
    return li_results

In [8]:
mse, rmse = evaluation_metrics(A, A_hat)

print(f'The mean squared error of all predictions is {mse:.3f}.')
print(f'The root mean squared error of all predictions is {rmse:.3f}.')

The mean squared error of all predictions is 0.295.
The root mean squared error of all predictions is 0.544.


Therefore, our matrix factorization algorithm is very good at predicting the existing entries. Let us see what the predictions were for the empty entries.

### Write Method to Predict the Zero Entries

In [9]:
# have a function predict the missing entries of our ground truth matrix

def mat_predict(A, A_hat, conv_to_int = True, zip_indices = False):
    # find zero indices of A (our missing entries that we are predicting)
    zero_ind = [(i,j) for i in range(A.shape[0]) for j in range(A.shape[1]) if A[i][j] == 0]
    
    # find predictions
    og_pred = [A_hat[i,j] for i,j in zero_ind]
    
    # WARNING: FOR VERY LARGE AND SPARSE MATRICES, I DO NOT RECOMMEND DOING THIS. JUST LOOK AT THE OUTPUT
    if zip_indices:
        for prediction, indices in zip(pred, zero_ind):
            print(f'The interaction at row, column {indices} is {prediction}.')
            
    # convert output to integers
    if conv_to_int:
        # if the rounded A_hat has entries greater than the max of A, then set it to floor, otherwise, round it to integer
        int_prediction = np.where(A_hat > np.max(A), np.floor(A_hat), np.round(A_hat, decimals=0))
        
        # return integer predictions at indices where 0's originally were in the ground truth A
        return [int_prediction[i,j] for i,j in zero_ind]
    else:
        # return the original floating point predictions from the algorithm output
        
        return og_pred

In [10]:
mat_predict(A, A_hat)

[1.0, 2.0, 2.0, 1.0, 1.0, 5.0, 3.0, 1.0, 2.0, 2.0]

### Write Method to Pipeline the Training and Evaluation of our Algorithm

We will incorporate all of the evaluation outputs of the algorithm in the following method:

In [11]:
# perform the matrix factorization, display the evaluation metrics, show time elapsed, return predictions
# this is a simple pipeline function for the evaluation of our algorithm

def evaluation_pipeline(A, U, V, epochs, latent_features, factorize_fn, flag_integer_ratings = False, print_divs=10):
        
    m, n = A.shape[0], A.shape[1]

    user_features = latent_features

    # compute number of entries in input matrix
    num_entries = A.shape[0] * A.shape[1]
    
    # compute density/sparsity of matrix
    density = np.count_nonzero(A)/num_entries
    
    sparsity = 1 - density    

    # perform factorization, record time
    start_time = t.default_timer()
    
    # perform matrix factorization
    U_hat, V_hat = factorize_fn(A, U, V, user_features, epochs, print_divs)

    end_time = t.default_timer()
    
    # factorization result
    A_hat = np.dot(U_hat, V_hat.T)
    
    # get run time
    time_elapsed = end_time - start_time
    
    # get evaluation metrics
    mse, rmse = evaluation_metrics(A, A_hat)
    
    # get predictions
    if flag_integer_ratings:
        ratings = mat_predict(A, A_hat)
    else:
        ratings = mat_predict(A, A_hat, conv_to_int=False)
    
    return {'Ratings': ratings, 'Metrics': (mse, rmse), 'Runtime': time_elapsed, 'Resulting Output': A_hat, 'Other Info': {'Sparsity': sparsity, 'Density': density, 'Input Dimension': (A.shape[0], A.shape[1])}}


In [12]:
# log everything out of evaluation_pipeline function so we don't have to write it

def call_eval_pipeline(dict_output, flag_output = True):
    
    predictions, metrics, runtime, A_hat, mat_facts = dict_output['Ratings'], dict_output['Metrics'], dict_output['Runtime'], dict_output['Resulting Output'], dict_output['Other Info']
    
    mse, rmse = metrics
    
    # facts about our original matrix (interaction table)
    sparsity, input_dim = mat_facts['Sparsity'], mat_facts['Input Dimension']
    if flag_output:
        print(f'Here is the resulting prediction matrix:\n{A_hat}\n')
        print(f'Here are the predicted ratings:\n{predictions}\n')
    else:
        pass
    
    print(f'The Mean Squared Error is {mse}.\nThe Root Mean Squared Error is {rmse}.\n')
    print(f'The Runtime of this algorithm of this {input_dim[0]} x {input_dim[1]}, {sparsity*100:.3f}% sparse matrix, is {runtime} seconds.')

**Non-Square Matrix Test Case**

In [13]:
A = np.array([
    [5,3,0,1],
    [3,2,0,0],
    [0,1,4,2],
    [1,0,0,5],
    [0,0,0,3]
])

m, n = A.shape[0], A.shape[1]

user_features = 2

np.random.seed(42)

U_1 = np.random.rand(m, user_features)
V_1 = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U_1)
print('V is:\n', V_1)

# perform factorization

dict_small_nonsq = evaluation_pipeline(A, U_1, V_1, 5000, 2, matrix_factorization)

call_eval_pipeline(dict_small_nonsq)


U is:
 [[0.37454012 0.95071431]
 [0.73199394 0.59865848]
 [0.15601864 0.15599452]
 [0.05808361 0.86617615]
 [0.60111501 0.70807258]]
V is:
 [[0.02058449 0.96990985]
 [0.83244264 0.21233911]
 [0.18182497 0.18340451]
 [0.30424224 0.52475643]]
Epoch 1 : Total Loss 76.548
Epoch 11 : Total Loss 75.321
Epoch 21 : Total Loss 74.076
Epoch 31 : Total Loss 72.817
Epoch 41 : Total Loss 71.544
Epoch 51 : Total Loss 70.260
Epoch 61 : Total Loss 68.968
Epoch 71 : Total Loss 67.669
Epoch 81 : Total Loss 66.366
Epoch 91 : Total Loss 65.062
Epoch 101 : Total Loss 63.759
Epoch 111 : Total Loss 62.460
Epoch 121 : Total Loss 61.169
Epoch 131 : Total Loss 59.887
Epoch 141 : Total Loss 58.617
Epoch 151 : Total Loss 57.363
Epoch 161 : Total Loss 56.126
Epoch 171 : Total Loss 54.909
Epoch 181 : Total Loss 53.715
Epoch 191 : Total Loss 52.546
Epoch 201 : Total Loss 51.404
Epoch 211 : Total Loss 50.290
Epoch 221 : Total Loss 49.206
Epoch 231 : Total Loss 48.154
Epoch 241 : Total Loss 47.135
Epoch 251 : Total Lo

Epoch 2681 : Total Loss 5.714
Epoch 2691 : Total Loss 5.579
Epoch 2701 : Total Loss 5.447
Epoch 2711 : Total Loss 5.317
Epoch 2721 : Total Loss 5.190
Epoch 2731 : Total Loss 5.065
Epoch 2741 : Total Loss 4.943
Epoch 2751 : Total Loss 4.823
Epoch 2761 : Total Loss 4.706
Epoch 2771 : Total Loss 4.592
Epoch 2781 : Total Loss 4.480
Epoch 2791 : Total Loss 4.371
Epoch 2801 : Total Loss 4.265
Epoch 2811 : Total Loss 4.161
Epoch 2821 : Total Loss 4.060
Epoch 2831 : Total Loss 3.962
Epoch 2841 : Total Loss 3.866
Epoch 2851 : Total Loss 3.773
Epoch 2861 : Total Loss 3.683
Epoch 2871 : Total Loss 3.595
Epoch 2881 : Total Loss 3.510
Epoch 2891 : Total Loss 3.428
Epoch 2901 : Total Loss 3.348
Epoch 2911 : Total Loss 3.270
Epoch 2921 : Total Loss 3.196
Epoch 2931 : Total Loss 3.123
Epoch 2941 : Total Loss 3.053
Epoch 2951 : Total Loss 2.985
Epoch 2961 : Total Loss 2.920
Epoch 2971 : Total Loss 2.857
Epoch 2981 : Total Loss 2.796
Epoch 2991 : Total Loss 2.738
Epoch 3001 : Total Loss 2.681
Epoch 3011

**Try a Large Matrix**

In [14]:
# np.random.seed(1)

# A = np.random.randint(0, 6, size=(100,100))

# m, n = A.shape[0], A.shape[1]

# user_features = 2

# U_2 = np.random.rand(m, user_features)
# V_2 = np.random.rand(n, user_features)

# # check entries of matrices

# print('U is:\n', U_2)
# print('V is:\n', V_2)

# # perform factorization

# U_hat, V_hat = matrix_factorization(A, U_2, V_2, user_features, 5000)

# A_hat = np.dot(U_hat, V_hat.T)

# print('U_hat is:\n', U_hat)
# print('V_hat is:\n', V_hat)

# ## check if the product results in something close to A
# print('A is:\n', A)
# print('A_hat is:\n', A_hat)


**Note: This takes a really long time to train and evaluate. It took over 15 minutes to train and evaluate a 100 by 100 matrix with 2 latent features. Most datasets that we will put in in a business situation will be WELL over that amount so this current implementation is not feasible for large datasets.**

## Matrix Factorization Implementation for Sparse Matrices

We will investigate how to perform matrix factorization for sparse matrices, as most matrices we will work with for the purpose of matrix factorization are large and sparse (if it were large but not sparse, we would have most of our information anyways, so this technique wouldn't be super useful).

### Sparse Matrices

The definition of a sparse matrix is very imprecise, but for our purposes, we define a sparse matrix as any matrix that stores a 0 (or any other value that represents the absence of meaningful data) in at most $\max(m,n)$ entries in an $m\times n$ matrix.

We can use scipy's csr_matrix class to get the nonzero elements of a sparse matrix. Let us use a toy example!

In [15]:
from scipy.sparse import csr_matrix

In [16]:
x = np.array([
    [0,1,0,0,0,0,0],
    [0,0,5,0,0,1,0],
    [0,3,0,0,0,0,1],
    [4,0,0,2,0,0,0]    
          ])

x

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 5, 0, 0, 1, 0],
       [0, 3, 0, 0, 0, 0, 1],
       [4, 0, 0, 2, 0, 0, 0]])

In [17]:
# print items in csr_matrix

for item in csr_matrix(x):
    print(item)

  (0, 1)	1
  (0, 2)	5
  (0, 5)	1
  (0, 1)	3
  (0, 6)	1
  (0, 0)	4
  (0, 3)	2


So a csr_matrix object only stores nonzero elements (the meaningful data entries) in memory!

This is important for our matrix factorization algorithm implementation here, because recall that **our loss computation only involves NONZERO elements** anyways! So we do not need any information about the matrix with the 0 entries!

We can pick out the nonzero indices of the matrix with **csr_matrix's nonzero method**.

In [18]:
csr_matrix(x).nonzero()

(array([0, 1, 1, 2, 2, 3, 3], dtype=int32),
 array([1, 2, 5, 1, 6, 0, 3], dtype=int32))

In [19]:
# index the nonzero elements through extracting the indices
li_ind = []

for x_ind, y_ind in zip(csr_matrix(x).nonzero()[0], csr_matrix(x).nonzero()[1]):
    li_ind.append((x_ind, y_ind))

li_ind

[(0, 1), (1, 2), (1, 5), (2, 1), (2, 6), (3, 0), (3, 3)]

In [20]:
# access all elements of our original array
for i, j in zip(csr_matrix(x).nonzero()[0], csr_matrix(x).nonzero()[1]):
    print(x[i,j])

1
5
1
3
1
4
2


### Matrix Factorization 2: Matrix Factorization for Sparse Matrices

We will be implementing the matrix factorization algorithm for sparse matrices here. We will see that this is a significant optimization over the first implementation.

In [21]:
# This function is different from the previous one because we do not need to traverse each array separately
# we isolate and extract all of the nonzero entries from the matrix in linear time (at worst O(n) for ground truth mxn)
# and then we only loop through epochs and the number of latent features

# ARGUMENTS
# A - ground truth matrix (sparse numpy array)
# U - product 1
# V - product 2
# number of features
# epochs - number of epochs to train
# lambda - learning rate
# beta - regularization parameter

def matrix_factorization_sparse(A, U, V, k, steps, lmbda=0.0002, beta = 0.02):
    # transpose V to make matrix multiplication work
    V = V.T
    
    # convert A, a sparse numpy array, into a sparse scipy matrix object
    sparse_A = csr_matrix(A)
    
    for epoch in range(steps):
        # initialize total loss
        e = 0
        for i, j in zip(sparse_A.nonzero()[0], sparse_A.nonzero()[1]):
            # component-wise error
            eij = A[i,j] - np.dot(U[i,:], V[:,j])
            # total error
            e += (eij**2)
            
            for K in range(k):
                # gradient descent step for each entry, with regularization term
                U[i,K] = U[i,K] + lmbda * (2 * eij * V[K,j] - beta*U[i,K])
                V[K,j] = V[K,j] + lmbda * (2 * eij * U[i,K] - beta*V[K,j])
                
                # sum up total loss with regularization terms
                e += 0.5 * beta * (U[i][K]**2 + V[K][j]**2)

        # give print output of training (print only every 10 epochs)
        if epoch % 10 == 0:
            print(f'Epoch {epoch + 1} : Total Loss {e:.3f}')
        if e < 0.001:
            break
    return U, V.T

### Test on a Small Example

In [22]:
A = np.array([
    [5,3,2,1,0],
    [3,0,4,2,1],
    [4,0,0,1,0],
    [0,0,2,4,0],
    [5,3,0,0,1]
])

m = A.shape[0]
n = A.shape[1]
user_features = 2

np.random.seed(24)

# generate matrices with entries of random numbers 0 - 1
U = np.random.rand(m, user_features)
V = np.random.rand(n, user_features)

dict_small_sparse1 = evaluation_pipeline(A, U, V, 5000, 2, matrix_factorization_sparse)

call_eval_pipeline(dict_small_sparse1)

Epoch 1 : Total Loss 99.448
Epoch 11 : Total Loss 96.944
Epoch 21 : Total Loss 94.384
Epoch 31 : Total Loss 91.770
Epoch 41 : Total Loss 89.107
Epoch 51 : Total Loss 86.400
Epoch 61 : Total Loss 83.655
Epoch 71 : Total Loss 80.879
Epoch 81 : Total Loss 78.078
Epoch 91 : Total Loss 75.261
Epoch 101 : Total Loss 72.436
Epoch 111 : Total Loss 69.612
Epoch 121 : Total Loss 66.798
Epoch 131 : Total Loss 64.003
Epoch 141 : Total Loss 61.238
Epoch 151 : Total Loss 58.510
Epoch 161 : Total Loss 55.830
Epoch 171 : Total Loss 53.205
Epoch 181 : Total Loss 50.644
Epoch 191 : Total Loss 48.155
Epoch 201 : Total Loss 45.745
Epoch 211 : Total Loss 43.419
Epoch 221 : Total Loss 41.183
Epoch 231 : Total Loss 39.040
Epoch 241 : Total Loss 36.995
Epoch 251 : Total Loss 35.049
Epoch 261 : Total Loss 33.205
Epoch 271 : Total Loss 31.461
Epoch 281 : Total Loss 29.818
Epoch 291 : Total Loss 28.275
Epoch 301 : Total Loss 26.829
Epoch 311 : Total Loss 25.479
Epoch 321 : Total Loss 24.220
Epoch 331 : Total Los

Epoch 2801 : Total Loss 5.571
Epoch 2811 : Total Loss 5.570
Epoch 2821 : Total Loss 5.569
Epoch 2831 : Total Loss 5.567
Epoch 2841 : Total Loss 5.566
Epoch 2851 : Total Loss 5.564
Epoch 2861 : Total Loss 5.563
Epoch 2871 : Total Loss 5.562
Epoch 2881 : Total Loss 5.561
Epoch 2891 : Total Loss 5.559
Epoch 2901 : Total Loss 5.558
Epoch 2911 : Total Loss 5.557
Epoch 2921 : Total Loss 5.556
Epoch 2931 : Total Loss 5.555
Epoch 2941 : Total Loss 5.553
Epoch 2951 : Total Loss 5.552
Epoch 2961 : Total Loss 5.551
Epoch 2971 : Total Loss 5.550
Epoch 2981 : Total Loss 5.549
Epoch 2991 : Total Loss 5.548
Epoch 3001 : Total Loss 5.547
Epoch 3011 : Total Loss 5.546
Epoch 3021 : Total Loss 5.545
Epoch 3031 : Total Loss 5.544
Epoch 3041 : Total Loss 5.544
Epoch 3051 : Total Loss 5.543
Epoch 3061 : Total Loss 5.542
Epoch 3071 : Total Loss 5.541
Epoch 3081 : Total Loss 5.540
Epoch 3091 : Total Loss 5.539
Epoch 3101 : Total Loss 5.538
Epoch 3111 : Total Loss 5.538
Epoch 3121 : Total Loss 5.537
Epoch 3131

### Test Runtime

We will test the runtime of the code we implemented for the sparse data.

We will test both methods for the same amount of epochs and latent features, learning rate, etc. on the same data.

**Regular Matrix Factorization**

In [23]:
start1 = t.default_timer()

U_hat, V_hat = matrix_factorization(A, U, V, user_features, 5000)

end1 = t.default_timer()

print('The runtime of the regular matrix factorization is ', end1 - start1, 'seconds.')

Epoch 1 : Total Loss 5.473
Epoch 11 : Total Loss 5.473
Epoch 21 : Total Loss 5.473
Epoch 31 : Total Loss 5.473
Epoch 41 : Total Loss 5.473
Epoch 51 : Total Loss 5.473
Epoch 61 : Total Loss 5.473
Epoch 71 : Total Loss 5.473
Epoch 81 : Total Loss 5.472
Epoch 91 : Total Loss 5.472
Epoch 101 : Total Loss 5.472
Epoch 111 : Total Loss 5.472
Epoch 121 : Total Loss 5.472
Epoch 131 : Total Loss 5.472
Epoch 141 : Total Loss 5.472
Epoch 151 : Total Loss 5.471
Epoch 161 : Total Loss 5.471
Epoch 171 : Total Loss 5.471
Epoch 181 : Total Loss 5.471
Epoch 191 : Total Loss 5.471
Epoch 201 : Total Loss 5.471
Epoch 211 : Total Loss 5.471
Epoch 221 : Total Loss 5.470
Epoch 231 : Total Loss 5.470
Epoch 241 : Total Loss 5.470
Epoch 251 : Total Loss 5.470
Epoch 261 : Total Loss 5.470
Epoch 271 : Total Loss 5.470
Epoch 281 : Total Loss 5.470
Epoch 291 : Total Loss 5.469
Epoch 301 : Total Loss 5.469
Epoch 311 : Total Loss 5.469
Epoch 321 : Total Loss 5.469
Epoch 331 : Total Loss 5.469
Epoch 341 : Total Loss 5.

Epoch 2861 : Total Loss 5.370
Epoch 2871 : Total Loss 5.369
Epoch 2881 : Total Loss 5.369
Epoch 2891 : Total Loss 5.368
Epoch 2901 : Total Loss 5.367
Epoch 2911 : Total Loss 5.366
Epoch 2921 : Total Loss 5.365
Epoch 2931 : Total Loss 5.364
Epoch 2941 : Total Loss 5.363
Epoch 2951 : Total Loss 5.362
Epoch 2961 : Total Loss 5.361
Epoch 2971 : Total Loss 5.360
Epoch 2981 : Total Loss 5.359
Epoch 2991 : Total Loss 5.358
Epoch 3001 : Total Loss 5.357
Epoch 3011 : Total Loss 5.356
Epoch 3021 : Total Loss 5.355
Epoch 3031 : Total Loss 5.354
Epoch 3041 : Total Loss 5.353
Epoch 3051 : Total Loss 5.352
Epoch 3061 : Total Loss 5.351
Epoch 3071 : Total Loss 5.350
Epoch 3081 : Total Loss 5.349
Epoch 3091 : Total Loss 5.347
Epoch 3101 : Total Loss 5.346
Epoch 3111 : Total Loss 5.345
Epoch 3121 : Total Loss 5.344
Epoch 3131 : Total Loss 5.343
Epoch 3141 : Total Loss 5.342
Epoch 3151 : Total Loss 5.341
Epoch 3161 : Total Loss 5.340
Epoch 3171 : Total Loss 5.338
Epoch 3181 : Total Loss 5.337
Epoch 3191

**Sparse Matrix Factorization**

In [24]:
start2 = t.default_timer()

U_hat_sparse, V_hat_sparse = matrix_factorization_sparse(A, U, V, user_features, 5000)

end2 = t.default_timer()

print('The runtime of the sparse matrix factorization is ', end2 - start2, 'seconds.')

Epoch 1 : Total Loss 4.899
Epoch 11 : Total Loss 4.895
Epoch 21 : Total Loss 4.891
Epoch 31 : Total Loss 4.886
Epoch 41 : Total Loss 4.882
Epoch 51 : Total Loss 4.878
Epoch 61 : Total Loss 4.873
Epoch 71 : Total Loss 4.869
Epoch 81 : Total Loss 4.864
Epoch 91 : Total Loss 4.860
Epoch 101 : Total Loss 4.856
Epoch 111 : Total Loss 4.851
Epoch 121 : Total Loss 4.847
Epoch 131 : Total Loss 4.842
Epoch 141 : Total Loss 4.838
Epoch 151 : Total Loss 4.833
Epoch 161 : Total Loss 4.828
Epoch 171 : Total Loss 4.824
Epoch 181 : Total Loss 4.819
Epoch 191 : Total Loss 4.815
Epoch 201 : Total Loss 4.810
Epoch 211 : Total Loss 4.805
Epoch 221 : Total Loss 4.800
Epoch 231 : Total Loss 4.796
Epoch 241 : Total Loss 4.791
Epoch 251 : Total Loss 4.786
Epoch 261 : Total Loss 4.781
Epoch 271 : Total Loss 4.776
Epoch 281 : Total Loss 4.772
Epoch 291 : Total Loss 4.767
Epoch 301 : Total Loss 4.762
Epoch 311 : Total Loss 4.757
Epoch 321 : Total Loss 4.752
Epoch 331 : Total Loss 4.747
Epoch 341 : Total Loss 4.

Epoch 2811 : Total Loss 3.173
Epoch 2821 : Total Loss 3.167
Epoch 2831 : Total Loss 3.162
Epoch 2841 : Total Loss 3.156
Epoch 2851 : Total Loss 3.151
Epoch 2861 : Total Loss 3.145
Epoch 2871 : Total Loss 3.140
Epoch 2881 : Total Loss 3.135
Epoch 2891 : Total Loss 3.129
Epoch 2901 : Total Loss 3.124
Epoch 2911 : Total Loss 3.119
Epoch 2921 : Total Loss 3.113
Epoch 2931 : Total Loss 3.108
Epoch 2941 : Total Loss 3.103
Epoch 2951 : Total Loss 3.098
Epoch 2961 : Total Loss 3.092
Epoch 2971 : Total Loss 3.087
Epoch 2981 : Total Loss 3.082
Epoch 2991 : Total Loss 3.077
Epoch 3001 : Total Loss 3.072
Epoch 3011 : Total Loss 3.067
Epoch 3021 : Total Loss 3.062
Epoch 3031 : Total Loss 3.057
Epoch 3041 : Total Loss 3.052
Epoch 3051 : Total Loss 3.047
Epoch 3061 : Total Loss 3.042
Epoch 3071 : Total Loss 3.037
Epoch 3081 : Total Loss 3.032
Epoch 3091 : Total Loss 3.027
Epoch 3101 : Total Loss 3.022
Epoch 3111 : Total Loss 3.017
Epoch 3121 : Total Loss 3.012
Epoch 3131 : Total Loss 3.007
Epoch 3141

There is not a significant difference on these, however, the $A$ that we used in the **above example is not sparse at all.** 14/25 entries are nonzero, so the sparsity is only 44%. **We will try this method on a significantly more sparse matrix.**

## Test on a Large Sparse Matrix

**This will be a 100 x 100 sparse matrix with 2 latent features, and it will be 99% sparse.**

In [25]:
# generate random sparse matrix
from scipy.sparse import random
from scipy.stats import randint
arr_rvs = randint(0, 6).rvs

x = random(100,100, density = 0.01, data_rvs = arr_rvs)

arr_100_100 = np.array(x.toarray())

arr_100_100

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
li_ind = [(i,j) for i in range(arr_100_100.shape[0]) for j in range(arr_100_100.shape[1]) if arr_100_100[i,j]!=0]

li_arr_100_100 = [arr_100_100[i,j] for i,j in li_ind]

li_arr_100_100, len(li_arr_100_100)

([4.0,
  5.0,
  4.0,
  2.0,
  1.0,
  2.0,
  1.0,
  5.0,
  1.0,
  1.0,
  1.0,
  1.0,
  3.0,
  4.0,
  3.0,
  5.0,
  5.0,
  2.0,
  5.0,
  4.0,
  1.0,
  3.0,
  3.0,
  5.0,
  2.0,
  1.0,
  4.0,
  4.0,
  2.0,
  2.0,
  1.0,
  1.0,
  3.0,
  4.0,
  3.0,
  3.0,
  2.0,
  3.0,
  2.0,
  3.0,
  1.0,
  3.0,
  1.0,
  2.0,
  5.0,
  5.0,
  5.0,
  3.0,
  4.0,
  4.0,
  3.0,
  1.0,
  4.0,
  5.0,
  3.0,
  5.0,
  5.0,
  1.0,
  5.0,
  1.0,
  1.0,
  4.0,
  1.0,
  4.0,
  2.0,
  2.0,
  2.0,
  3.0,
  3.0,
  1.0,
  4.0,
  2.0,
  2.0,
  1.0,
  3.0,
  1.0,
  3.0,
  4.0,
  4.0,
  2.0,
  1.0,
  1.0,
  3.0,
  3.0],
 84)

### Run the Sparse Matrix Factorization on our Sparse Matrix

In [27]:
np.random.seed(1)

m, n = arr_100_100.shape[0], arr_100_100.shape[1]

user_features = 2

U_3 = np.random.rand(m, user_features)
V_3 = np.random.rand(n, user_features)

# run through pipeline

dict_100_sparse = evaluation_pipeline(arr_100_100, U_3, V_3, 5000, 2, matrix_factorization_sparse)

call_eval_pipeline(dict_100_sparse, flag_output = False)

Epoch 1 : Total Loss 598.880
Epoch 11 : Total Loss 590.512
Epoch 21 : Total Loss 582.027
Epoch 31 : Total Loss 573.428
Epoch 41 : Total Loss 564.718
Epoch 51 : Total Loss 555.900
Epoch 61 : Total Loss 546.979
Epoch 71 : Total Loss 537.961
Epoch 81 : Total Loss 528.851
Epoch 91 : Total Loss 519.656
Epoch 101 : Total Loss 510.385
Epoch 111 : Total Loss 501.045
Epoch 121 : Total Loss 491.646
Epoch 131 : Total Loss 482.197
Epoch 141 : Total Loss 472.709
Epoch 151 : Total Loss 463.193
Epoch 161 : Total Loss 453.660
Epoch 171 : Total Loss 444.122
Epoch 181 : Total Loss 434.591
Epoch 191 : Total Loss 425.079
Epoch 201 : Total Loss 415.598
Epoch 211 : Total Loss 406.161
Epoch 221 : Total Loss 396.780
Epoch 231 : Total Loss 387.465
Epoch 241 : Total Loss 378.230
Epoch 251 : Total Loss 369.084
Epoch 261 : Total Loss 360.038
Epoch 271 : Total Loss 351.103
Epoch 281 : Total Loss 342.286
Epoch 291 : Total Loss 333.597
Epoch 301 : Total Loss 325.042
Epoch 311 : Total Loss 316.630
Epoch 321 : Total L

Epoch 2711 : Total Loss 9.554
Epoch 2721 : Total Loss 9.518
Epoch 2731 : Total Loss 9.483
Epoch 2741 : Total Loss 9.448
Epoch 2751 : Total Loss 9.413
Epoch 2761 : Total Loss 9.379
Epoch 2771 : Total Loss 9.345
Epoch 2781 : Total Loss 9.312
Epoch 2791 : Total Loss 9.279
Epoch 2801 : Total Loss 9.246
Epoch 2811 : Total Loss 9.213
Epoch 2821 : Total Loss 9.181
Epoch 2831 : Total Loss 9.150
Epoch 2841 : Total Loss 9.118
Epoch 2851 : Total Loss 9.087
Epoch 2861 : Total Loss 9.056
Epoch 2871 : Total Loss 9.026
Epoch 2881 : Total Loss 8.996
Epoch 2891 : Total Loss 8.966
Epoch 2901 : Total Loss 8.936
Epoch 2911 : Total Loss 8.907
Epoch 2921 : Total Loss 8.878
Epoch 2931 : Total Loss 8.849
Epoch 2941 : Total Loss 8.821
Epoch 2951 : Total Loss 8.793
Epoch 2961 : Total Loss 8.765
Epoch 2971 : Total Loss 8.737
Epoch 2981 : Total Loss 8.710
Epoch 2991 : Total Loss 8.683
Epoch 3001 : Total Loss 8.656
Epoch 3011 : Total Loss 8.629
Epoch 3021 : Total Loss 8.603
Epoch 3031 : Total Loss 8.577
Epoch 3041

### Run Regular Matrix Factorization on Sparse Matrix

Just for comparison, let us check how much more efficient our sparse matrix factorization is compared to the regular method.

In [28]:
np.random.seed(1)

m, n = arr_100_100.shape[0], arr_100_100.shape[1]

user_features = 2

U_3 = np.random.rand(m, user_features)
V_3 = np.random.rand(n, user_features)

dict_100_regular = evaluation_pipeline(arr_100_100, U_3, V_3, 5000, 2, matrix_factorization)

call_eval_pipeline(dict_100_regular, flag_output = False)

Epoch 1 : Total Loss 598.166
Epoch 11 : Total Loss 589.788
Epoch 21 : Total Loss 581.294
Epoch 31 : Total Loss 572.685
Epoch 41 : Total Loss 563.966
Epoch 51 : Total Loss 555.139
Epoch 61 : Total Loss 546.209
Epoch 71 : Total Loss 537.182
Epoch 81 : Total Loss 528.064
Epoch 91 : Total Loss 518.862
Epoch 101 : Total Loss 509.584
Epoch 111 : Total Loss 500.238
Epoch 121 : Total Loss 490.834
Epoch 131 : Total Loss 481.381
Epoch 141 : Total Loss 471.889
Epoch 151 : Total Loss 462.370
Epoch 161 : Total Loss 452.835
Epoch 171 : Total Loss 443.296
Epoch 181 : Total Loss 433.765
Epoch 191 : Total Loss 424.254
Epoch 201 : Total Loss 414.775
Epoch 211 : Total Loss 405.340
Epoch 221 : Total Loss 395.962
Epoch 231 : Total Loss 386.653
Epoch 241 : Total Loss 377.422
Epoch 251 : Total Loss 368.283
Epoch 261 : Total Loss 359.244
Epoch 271 : Total Loss 350.317
Epoch 281 : Total Loss 341.508
Epoch 291 : Total Loss 332.828
Epoch 301 : Total Loss 324.284
Epoch 311 : Total Loss 315.882
Epoch 321 : Total L

Epoch 2661 : Total Loss 9.728
Epoch 2671 : Total Loss 9.691
Epoch 2681 : Total Loss 9.653
Epoch 2691 : Total Loss 9.616
Epoch 2701 : Total Loss 9.580
Epoch 2711 : Total Loss 9.544
Epoch 2721 : Total Loss 9.508
Epoch 2731 : Total Loss 9.473
Epoch 2741 : Total Loss 9.438
Epoch 2751 : Total Loss 9.403
Epoch 2761 : Total Loss 9.369
Epoch 2771 : Total Loss 9.335
Epoch 2781 : Total Loss 9.302
Epoch 2791 : Total Loss 9.269
Epoch 2801 : Total Loss 9.236
Epoch 2811 : Total Loss 9.204
Epoch 2821 : Total Loss 9.172
Epoch 2831 : Total Loss 9.140
Epoch 2841 : Total Loss 9.109
Epoch 2851 : Total Loss 9.078
Epoch 2861 : Total Loss 9.047
Epoch 2871 : Total Loss 9.017
Epoch 2881 : Total Loss 8.987
Epoch 2891 : Total Loss 8.957
Epoch 2901 : Total Loss 8.927
Epoch 2911 : Total Loss 8.898
Epoch 2921 : Total Loss 8.869
Epoch 2931 : Total Loss 8.841
Epoch 2941 : Total Loss 8.812
Epoch 2951 : Total Loss 8.784
Epoch 2961 : Total Loss 8.756
Epoch 2971 : Total Loss 8.729
Epoch 2981 : Total Loss 8.702
Epoch 2991

This is almost a 900% improvement in runtime!

### Run on a 1000 x 1000 sparse matrix

In [29]:
# 1k x 1k sparse matrix initialization

arr_rvs_2 = randint(0, 6).rvs

x_1k = random(1000,1000, density = 0.01, data_rvs = arr_rvs)

arr_1k = np.array(x_1k.toarray())

# check sparsity of matrix

li_nonzero_1k = [(i,j) for i in range(arr_1k.shape[0]) for j in range(arr_1k.shape[1]) if arr_1k[i,j]!=0]

sparsity_1k = len(li_nonzero_1k)/(arr_1k.shape[0]*arr_1k.shape[1])

print(f'The sparsity of our random 1000 x 1000 interaction table is {1 - sparsity_1k}.')

The sparsity of our random 1000 x 1000 interaction table is 0.991658.


In [30]:
np.random.seed(23)

m, n = arr_1k.shape[0], arr_1k.shape[1]

user_features = 2

U_4 = np.random.rand(m, user_features)
V_4 = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U_4)
print('V is:\n', V_4)

dict_1k_sparse = evaluation_pipeline(arr_1k, U_4, V_4, 5000, 2, matrix_factorization_sparse)

call_eval_pipeline(dict_1k_sparse, flag_output = False)

U is:
 [[0.51729788 0.9469626 ]
 [0.76545976 0.28239584]
 [0.22104536 0.68622209]
 ...
 [0.81725319 0.69902473]
 [0.63405118 0.1848986 ]
 [0.65351892 0.97255327]]
V is:
 [[0.17041509 0.77499716]
 [0.89300173 0.66987814]
 [0.960109   0.99091208]
 ...
 [0.87679857 0.42425804]
 [0.12411982 0.29636888]
 [0.73663472 0.99557962]]
Epoch 1 : Total Loss 70857.643
Epoch 11 : Total Loss 66719.825
Epoch 21 : Total Loss 62222.614
Epoch 31 : Total Loss 57446.104
Epoch 41 : Total Loss 52505.498
Epoch 51 : Total Loss 47542.677
Epoch 61 : Total Loss 42711.010
Epoch 71 : Total Loss 38156.121
Epoch 81 : Total Loss 33997.255
Epoch 91 : Total Loss 30314.089
Epoch 101 : Total Loss 27141.978
Epoch 111 : Total Loss 24475.682
Epoch 121 : Total Loss 22279.020
Epoch 131 : Total Loss 20496.831
Epoch 141 : Total Loss 19066.065
Epoch 151 : Total Loss 17924.180
Epoch 161 : Total Loss 17014.357
Epoch 171 : Total Loss 16287.974
Epoch 181 : Total Loss 15705.124
Epoch 191 : Total Loss 15233.998
Epoch 201 : Total Loss 14

Epoch 2391 : Total Loss 8501.774
Epoch 2401 : Total Loss 8496.257
Epoch 2411 : Total Loss 8490.787
Epoch 2421 : Total Loss 8485.362
Epoch 2431 : Total Loss 8479.983
Epoch 2441 : Total Loss 8474.647
Epoch 2451 : Total Loss 8469.356
Epoch 2461 : Total Loss 8464.108
Epoch 2471 : Total Loss 8458.903
Epoch 2481 : Total Loss 8453.740
Epoch 2491 : Total Loss 8448.618
Epoch 2501 : Total Loss 8443.537
Epoch 2511 : Total Loss 8438.496
Epoch 2521 : Total Loss 8433.496
Epoch 2531 : Total Loss 8428.535
Epoch 2541 : Total Loss 8423.612
Epoch 2551 : Total Loss 8418.728
Epoch 2561 : Total Loss 8413.881
Epoch 2571 : Total Loss 8409.072
Epoch 2581 : Total Loss 8404.300
Epoch 2591 : Total Loss 8399.564
Epoch 2601 : Total Loss 8394.863
Epoch 2611 : Total Loss 8390.198
Epoch 2621 : Total Loss 8385.568
Epoch 2631 : Total Loss 8380.973
Epoch 2641 : Total Loss 8376.411
Epoch 2651 : Total Loss 8371.884
Epoch 2661 : Total Loss 8367.389
Epoch 2671 : Total Loss 8362.928
Epoch 2681 : Total Loss 8358.498
Epoch 2691

Epoch 4881 : Total Loss 7792.302
Epoch 4891 : Total Loss 7790.610
Epoch 4901 : Total Loss 7788.924
Epoch 4911 : Total Loss 7787.244
Epoch 4921 : Total Loss 7785.571
Epoch 4931 : Total Loss 7783.904
Epoch 4941 : Total Loss 7782.244
Epoch 4951 : Total Loss 7780.589
Epoch 4961 : Total Loss 7778.942
Epoch 4971 : Total Loss 7777.300
Epoch 4981 : Total Loss 7775.665
Epoch 4991 : Total Loss 7774.037
The Mean Squared Error is 0.8449173567571847.
The Root Mean Squared Error is 0.9191938624453412.

The Runtime of this algorithm of this 1000 x 1000, 99.166% sparse matrix, is 1070.5518378000706 seconds.


## Improvements to Sparse Matrix Factorization

We will make some improvements to our sparse matrix factorization. The main improvement we can make in our code is that we can try to vectorize as many computations as possible, and take out as many computations out of the loops as we can.

### Matrix Factorization 3: Vectorize Sparse Matrix Factorization

In [31]:
# This function is different from the previous one because we do not need to traverse each array separately
# we isolate and extract all of the nonzero entries from the matrix in linear time (at worst O(n) for ground truth mxn)
# and then we only loop through epochs and the number of latent features

# UPDATE: This will also be vectorized, we will make notes of the changes in comments below

# ARGUMENTS
# A - ground truth matrix (sparse numpy array)
# U - product 1
# V - product 2
# number of features
# epochs - number of epochs to train
# lambda - learning rate
# beta - regularization parameter

def matrix_factorization_sparse_vectorized(A, U, V, k, steps, lmbda=0.0002, beta = 0.02, print_divs=10):
    # transpose V to make matrix multiplication work
    V = V.T
    
    # convert A, a sparse numpy array, into a sparse scipy matrix object
    sparse_A = csr_matrix(A)
    
    # start epochs
    for epoch in range(steps):
        # initialize total loss
        e = 0
        # get matrix of losses ahead of time and only look at the nonzero entries of A
        A_hat = np.matmul(U,V)
        err = A - A_hat
        
        # parse through nonzero indices of A
        for i, j in zip(sparse_A.nonzero()[0], sparse_A.nonzero()[1]):
            # component-wise error
            eij = err[i,j]
            # total error
            e += eij**2
                        
            for K in range(k):
                # gradient descent step for each entry, with regularization term, vectorized
                U[i,K], V[K,j] = U[i,K] + lmbda * (2 * eij * V[K,j] - beta*U[i,K]), V[K,j] + lmbda * (2 * eij * U[i,K] - beta*V[K,j])
                e += 0.5 * beta * (U[i][K]**2 + V[K][j]**2)
        
        # give print output of training (print only every 10 epochs)
        if epoch % print_steps == 0:
            print(f'Epoch {epoch + 1} : Total Loss {e:.3f}')
        if e < 0.001:
            break
    return U, V.T

In [32]:
np.random.seed(23)

m, n = arr_1k.shape[0], arr_1k.shape[1]

user_features = 2

U_4 = np.random.rand(m, user_features)
V_4 = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U_4)
print('V is:\n', V_4)

dict_1k_sparse = evaluation_pipeline(arr_1k, U_4, V_4, 5000, 2, matrix_factorization_sparse_vectorized, print_divs=100)

call_eval_pipeline(dict_1k_sparse, flag_output = False)

U is:
 [[0.51729788 0.9469626 ]
 [0.76545976 0.28239584]
 [0.22104536 0.68622209]
 ...
 [0.81725319 0.69902473]
 [0.63405118 0.1848986 ]
 [0.65351892 0.97255327]]
V is:
 [[0.17041509 0.77499716]
 [0.89300173 0.66987814]
 [0.960109   0.99091208]
 ...
 [0.87679857 0.42425804]
 [0.12411982 0.29636888]
 [0.73663472 0.99557962]]
Epoch 1 : Total Loss 71020.908
Epoch 11 : Total Loss 66894.562
Epoch 21 : Total Loss 62403.561
Epoch 31 : Total Loss 57626.281
Epoch 41 : Total Loss 52676.891
Epoch 51 : Total Loss 47697.317
Epoch 61 : Total Loss 42842.240
Epoch 71 : Total Loss 38259.675
Epoch 81 : Total Loss 34071.817
Epoch 91 : Total Loss 30361.167
Epoch 101 : Total Loss 27165.197
Epoch 111 : Total Loss 24479.792
Epoch 121 : Total Loss 22268.942
Epoch 131 : Total Loss 20476.971
Epoch 141 : Total Loss 19039.958
Epoch 151 : Total Loss 17894.423
Epoch 161 : Total Loss 16982.711
Epoch 171 : Total Loss 16255.546
Epoch 181 : Total Loss 15672.558
Epoch 191 : Total Loss 15201.633
Epoch 201 : Total Loss 14

Epoch 2391 : Total Loss 8481.004
Epoch 2401 : Total Loss 8475.490
Epoch 2411 : Total Loss 8470.022
Epoch 2421 : Total Loss 8464.599
Epoch 2431 : Total Loss 8459.221
Epoch 2441 : Total Loss 8453.888
Epoch 2451 : Total Loss 8448.599
Epoch 2461 : Total Loss 8443.353
Epoch 2471 : Total Loss 8438.150
Epoch 2481 : Total Loss 8432.989
Epoch 2491 : Total Loss 8427.869
Epoch 2501 : Total Loss 8422.790
Epoch 2511 : Total Loss 8417.751
Epoch 2521 : Total Loss 8412.753
Epoch 2531 : Total Loss 8407.793
Epoch 2541 : Total Loss 8402.873
Epoch 2551 : Total Loss 8397.990
Epoch 2561 : Total Loss 8393.146
Epoch 2571 : Total Loss 8388.338
Epoch 2581 : Total Loss 8383.567
Epoch 2591 : Total Loss 8378.833
Epoch 2601 : Total Loss 8374.134
Epoch 2611 : Total Loss 8369.471
Epoch 2621 : Total Loss 8364.843
Epoch 2631 : Total Loss 8360.249
Epoch 2641 : Total Loss 8355.689
Epoch 2651 : Total Loss 8351.162
Epoch 2661 : Total Loss 8346.669
Epoch 2671 : Total Loss 8342.209
Epoch 2681 : Total Loss 8337.781
Epoch 2691

Epoch 4881 : Total Loss 7771.263
Epoch 4891 : Total Loss 7769.572
Epoch 4901 : Total Loss 7767.887
Epoch 4911 : Total Loss 7766.209
Epoch 4921 : Total Loss 7764.537
Epoch 4931 : Total Loss 7762.871
Epoch 4941 : Total Loss 7761.212
Epoch 4951 : Total Loss 7759.559
Epoch 4961 : Total Loss 7757.912
Epoch 4971 : Total Loss 7756.272
Epoch 4981 : Total Loss 7754.638
Epoch 4991 : Total Loss 7753.011
The Mean Squared Error is 0.845015220602158.
The Root Mean Squared Error is 0.9192470944213846.

The Runtime of this algorithm of this 1000 x 1000, 99.166% sparse matrix, is 922.500383000006 seconds.


**Note: The runtime of the algorithm is very inconsistent, this particular algorithm has clocked as low as 5 minutes to train and evaluate fully. However, our vectorized operation is, for a fact, much quicker than the unvectorized version, nearly 2.5 minutes quicker, in fact.**

### Complexity Analysis

The time complexity of the matrix factorization methods are not entirely clear as it depends on the cases that we are looking at.

Let us lay down some definitions:

**Sparsity (s)**:

$s$ is a parameter denoting the proportion of 0's in a matrix compared to the total number of entries. For an $m\times n$ interaction matrix, $A$, we see that the **sparsity of $A$** is:

$$s = \frac{\mathrm{Number \; of \; 0's}}{nm}$$

**Density (d)**:

Likewise, $d$ is a parameter denoting the proportion of nonzero's compared to the total number of entries. For $A$ indicated above, the **density of $A$** is:

$$d = \frac{\mathrm{Number \;of\;nonzero's}}{nm}$$

Sparsity and density are related by the following expression:

$$s = 1 - d$$

(rearrange to see density in terms of sparsity)

Now onto our analysis:

1) **Case 1: Dense Matrices**

There is no clear definition, but for our purposes, we consider a **dense matrix**, $A$ (assume $m\times n$) as one in which there is a **density** of at least $\frac{nm}{2}$, where the nonzero entries are distributed with little bias across all rows and columns.

Let $k$ denote the **number of latent features**, $n$ denote the **number of columns**, and $m$ the **number of rows**. Assume that our number of latent features is *fixed*, and is far smaller than the dimensions of our interaction table.  Assume that our training loop persists for $e$ **epochs** and that $e$ is *fixed* (in practice, the number of epochs can get large, but, in principle, it is fixed before we run our training loop).

Let $N := \max(m,n)$. For a general $m\times n$ ground truth, interaction matrix, $A$, the *regular matrix factorization* implementation has a big $\mathcal{O}$ time complexity of $\mathcal{O}(ekmn)$. In our case, since the latent features and epochs are fixed in size (the dimensions of the matrix are features of our data and model), we can consider $k << N$, $e << N$, and $m$ and $n$ to be arbitrarily large, and we see that the worst-case runtime is actually just $\mathcal{O}(N^{2})$.

Now look at the *sparse matrix factorization* implementation, note that this is no different than the *regular matrix factorization* as when we isolate the indices corresponding to nonzero entries, we, essentially, must traverse at least $0.5nm$ entries in $A$. Therefore, the time complexity of the sparse MF is just $\mathcal{O}(N^{2})$.

2) **Case 2: Sparse Matrices**

We consider a **sparse matrix** as one in which the **sparsity** is **at least** $1 - 1/\min(n,m)$ (i.e. the number of nonzero entries is approximately the number of rows or the number of columns). With notation above, we see that the worst-case runtime for the *sparse matrix factorization* is $\mathcal{O}(N)$, and the worst-case runtime for the *regular matrix factorization* is $\mathcal{O}(N^{2})$, and to see this, just recognize that $e$ and $k$ are fixed as above. 

In our *sparse matrix factorization*, since **we only traverse nonzero entries of the interaction table**, we are only running our training loop on at most $\max(n,m)$ entries. Therefore, since $k$ and $e$ are fixed, we get a time complexity of $\sim \mathcal{O}(N)$.

In our *regular matrix factorization*, since **we must traverse ALL entries of the interaction table**, we are running our training loop for $nm$ entries as we **must traverse the zero entries in order to check that they are zero**. Therefore, we have a time complexity of $\sim \mathcal{O}(N^{2})$.

### Why is Sparsity Important?

Sparsity is an important assumption about our input data that is actually true for the business problem. Oftentimes, when we are trying to analyze interactions, we will have many entries missing. Most people either do not leave ratings or the ratings are often not present due to some preprocessing method (like one-hot encoding). A sparse matrix is a very realistic and conservative assumption to make when we are trying to generate recommendations.

Not only does sparsity optimize our method above, but it is an assumption that often holds in the business problem where matrix factorization is relevant.